In [20]:
import os
import cv2
import pandas
import numpy as np
import matplotlib.pyplot as plt

def extract_image(f):
    im = cv2.imread(f)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2YUV)
    im = im[:,:,0]
    mn, mx = np.min(im), np.max(im)
    a, b = 0., 1.
    im = a + ((im-mn)/(mx-mn))*(b-a)
    return im

def process_db(dir):
    with open(os.path.join(dir, 'driving_log.csv'), 'r') as f:
        data = pandas.read_csv(f, header=0)
        res = np.asarray(data.values)
        c, l, r, st, t, b, sp = np.split(res, 7, axis=1)
        c = np.array([extract_image(os.path.join(dir, p)) for p in c.squeeze()])
        return c, st.squeeze()
        
c, st = process_db('../data')
print(c.shape)
%matplotlib inline
#plt.imshow(c[20], cmap='gray')
        

(8036, 160, 320)
[[ 0.56862745  0.57647059  0.6        ...,  0.25882353  0.38431373
   0.3372549 ]
 [ 0.63921569  0.59215686  0.26666667 ...,  0.67058824  0.22352941
   0.70588235]
 [ 0.54117647  0.58431373  0.43137255 ...,  0.69019608  0.33333333
   0.66666667]
 ..., 
 [ 0.54901961  0.50980392  0.47058824 ...,  0.54509804  0.56078431
   0.57254902]
 [ 0.5372549   0.49803922  0.45882353 ...,  0.56078431  0.53333333
   0.50588235]
 [ 0.45490196  0.42745098  0.4        ...,  0.51372549  0.56078431
   0.61176471]]


In [22]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

c, st = shuffle(c, st)
st = st.astype(np.float32)
c = np.reshape(c, [-1, 160, 320, 1])
print(c.shape)


X_train, y_train = c[:200], st[:200]



(8036, 160, 320, 1)


In [ ]:
plt.bar(y_train, X_train)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.wrappers.scikit_learn import KerasRegressor

from keras.layers.pooling import MaxPooling2D
import numpy as np
from random import random

def my_model():
    model = Sequential()
    model.add(Convolution2D(128, 5, 5, border_mode='same', activation='relu', name='c1', input_shape=(160, 320, 1)))
    model.add(MaxPooling2D((2,2)))
    model.add(Convolution2D(256, 5, 5, border_mode='same', activation='relu', name='c2'))
    model.add(MaxPooling2D((2,2)))
    model.add(Convolution2D(512, 5, 5, border_mode='same', activation='relu', name='c3'))
    model.add(MaxPooling2D((2,2)))

    model.add(Flatten())
    model.add(Dense(256, name='d1',activation='relu'))
    model.add(Dense(128, name='d2',activation='linear'))
    model.add(Dense(64, name='d3',activation='relu'))
    model.add(Dense(1, name='d4', activation='linear'))
    
    return model


model = my_model()
model.compile('adam', 'mse', ['mse'])
history = model.fit(X_train, y_train, batch_size=128, nb_epoch=30, verbose=1, validation_split=0.2)
y_pred = model.predict(X_train)
print(y_pred)




Train on 160 samples, validate on 40 samples
Epoch 1/30
160/160 [==============================] - 331s - loss: 3374.9421 - mean_squared_error: 3374.9421 - val_loss: 35.6115 - val_mean_squared_error: 35.6115
Epoch 2/30
160/160 [==============================] - 326s - loss: 28.4095 - mean_squared_error: 28.4095 - val_loss: 0.5004 - val_mean_squared_error: 0.5004
Epoch 3/30
160/160 [==============================] - 322s - loss: 0.4179 - mean_squared_error: 0.4179 - val_loss: 0.0473 - val_mean_squared_error: 0.0473
Epoch 4/30
160/160 [==============================] - 318s - loss: 0.0501 - mean_squared_error: 0.0501 - val_loss: 0.0061 - val_mean_squared_error: 0.0061
Epoch 5/30
160/160 [==============================] - 320s - loss: 0.0223 - mean_squared_error: 0.0223 - val_loss: 0.0073 - val_mean_squared_er